**Tensorflow Play**

Getting familiar with Tensorflow by developing a simple classifier.

In [1]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("../src")
import utils as u

**Getting the Data**

In [2]:
X_train, X_test, X_valid, y_train, y_test, y_valid = u.get_fashion_mnist_data()

**Create a Model Wrapper**

Useful to take advantage of Scikit's `GridSearchCV` and/or `RandomizedSearchCV`

[Keras Wrapper](https://www.tensorflow.org/api_docs/python/tf/keras/wrappers/scikit_learn/KerasClassifier)

In [3]:
def build_model(n_hidden=2, n_neurons=300, learning_rate=3e-3):
    # Start with a simple sequential model
    model = keras.models.Sequential()
    
    # Flatten the input
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    
    # Add dense layers
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    
    # Add Final Layer
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    # Add Optimizer & Compile model
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    
    return model

In [4]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

The `keras_reg` object can now be used like a regular Scikit classifier.

In [ ]:
%%time
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": [50,100,200,300],
    "learning_rate": [0.0003,0.003,0.03],
}

checkpoint_cb = keras.callbacks.ModelCheckpoint("../models/classification_hyperparameter_tunning.h5",
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                  monitor='accuracy')
tensorboard_cb = keras.callbacks.TensorBoard(u.get_run_logdir())

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)

rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

`Wall time: 29min 20s`

In the code above, `RandomizedSearchCV` uses K-fold cross-validation, so it does not use `X_valid` and `y_valid`, which are only used for early stopping.

Given that they are Scikit's objects, we can access their regular attributes.

In [6]:
rnd_search_cv.best_params_

{'n_neurons': 300, 'n_hidden': 3, 'learning_rate': 0.03}

In [7]:
rnd_search_cv.best_score_

0.8878727275371552

In [10]:
model = rnd_search_cv.best_estimator_.model

In [11]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_78 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_79 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_80 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_81 (Dense)             (None, 10)                3010      
Total params: 419,110
Trainable params: 419,110
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.4049 - accuracy: 0.8801


[0.40488818287849426, 0.8801000118255615]

Slight improvement over the model trained previously.